In [10]:
!pip install -U ultralytics


In [11]:
!pip install --upgrade ultralytics


In [21]:
!nvidia-smi


Tue Nov 25 09:09:32 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.54.15              Driver Version: 550.54.15      CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  Tesla T4                       Off |   00000000:00:04.0 Off |                    0 |
| N/A   77C    P0             36W /   70W |   15092MiB /  15360MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

In [12]:
import torch
print(torch.cuda.is_available())  # should be True


True


In [13]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [14]:
import torch
from ultralytics import YOLO
import numpy as np
from pathlib import Path
import os
import sys
import cv2

In [22]:
model = YOLO("yolo11n-seg.pt")


In [23]:

dataset_path = Path(r'/content/drive/MyDrive/all data/Etioplast_New_226_Image/Dataset2/data.yaml')
if not dataset_path.exists():
    print(f"Error: data.yaml not found at {dataset_path}")
    print("Please run prepare_dataset.py first")
    sys.exit(1)

In [26]:
try:
    print("🚀 Starting training...")
    print(f"📁 Dataset config: {dataset_path.absolute()}")

    # results = model.train(
    #     data=str(dataset_path),             # path to dataset YAML
    #     epochs=100,                         # you can increase if needed
    #     imgsz=1024,                         # higher resolution for organelles
    #     batch=16,                            # reduce if OOM on GPU
    #     name='etioplast_segmentation_yolo11s-seg',   # experiment name
    #     project='EtioplastTrain',           # main folder in /runs
    #     device='cuda' if torch.cuda.is_available() else 'cpu',
    #     save=True,
    #     save_period=10,                     # save every 10 epochs
    #     patience=20,                        # early stopping
    #     verbose=True
    # )
    results = model.train(
          data=str(dataset_path),
          epochs=100,
          imgsz=1024,                     # ↓ from 1024
          batch=8,                       # ↓ from 16, lower if still OOM (try 2 or even 1)
          workers=1,                     # fewer dataloader workers
          name='etioplast_segmentation_yolo11n-seg',
          project='EtioplastTrain',
          device=0,                      # let Ultralytics use cuda:0
          save=True,
          save_period=10,
          patience=20,
          verbose=True,
          cache=False,                   # keep it off to save RAM/VRAM
      )


    print("✅ Training completed successfully!")

except Exception as e:
    print(f"❌ Error during training: {str(e)}")
    sys.exit(1)

🚀 Starting training...
📁 Dataset config: /content/drive/MyDrive/all data/Etioplast_New_226_Image/Dataset2/data.yaml
Ultralytics 8.3.231 🚀 Python-3.12.12 torch-2.9.0+cu126 CUDA:0 (Tesla T4, 15095MiB)
engine/trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=8, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, compile=False, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=/content/drive/MyDrive/all data/Etioplast_New_226_Image/Dataset2/data.yaml, degrees=0.0, deterministic=True, device=0, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=100, erasing=0.4, exist_ok=False, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=1024, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.01, lrf=0.01, mask_ratio=4, max_det=300, mixup=0.0, mode=train, model=yolo11n-seg.pt, momentum=0.937, mosaic=1.0

In [ ]:
model.predict(source=r"/content/drive/MyDrive/all data/Etioplast_New_226_Image/Dataset2/images/val/bean__11.png", save=True, conf=0.25)


In [ ]:
#Function to predict on new images
def predict_image(model, image_path):
    results = model(image_path)
    print(results)
    return results[0]

In [ ]:
def plot_predictions(image_path, results):
    plt.figure(figsize=(10, 10))
    plt.imshow(results[0].plot())  # Access the first item in the results list
    plt.axis('off')
    plt.show()